In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_request = f'''https://www.hondapartsnow.com/genuine/{crawler_series['OE'].replace(' ', '').replace('-', '')}.html'''

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 404:
                    break
                elif resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    dict_ = json.loads(html.xpath('//script[@id="initialState"]/text()')[0].split('window.__INITIAL_STORE__ =')[1].replace('undefined', '"undefined"'))
                    if dict_:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if resp.status_code == 404:
            raise

        # = = = = = = = = = = = = = = =

        h1 = html.xpath('//h1[@class="pn-detail-h1"]')[0]
        title = etree.tostring(h1).decode('utf-8')
        title = re.sub(r'<.*?>', '', title)
        title = re.sub(r' +', ' ', title.strip())
        title = html_lib.unescape(title)

        # = = = = = = = = = = = = = = =

        subtitle = html.xpath('//p[@class="pn-detail-sub-desc"]/text()')[0].strip()

        # = = = = = = = = = = = = = = =

        part_number = dict_['partNumber']['partInfo']['partNumber'].strip()

        # = = = = = = = = = = = = = = =

        sku = dict_['partNumber']['partInfo']['partNumberAbbr'].strip()

        # = = = = = = = = = = = = = = =

        type_ = dict_['partNumber']['partInfo']['mainDesc'].strip()

        # = = = = = = = = = = = = = = =

        description = dict_['partNumber']['partInfo']['subDesc'].strip()

        # = = = = = = = = = = = = = = =

        list_specification, dict_count = dict_['partNumber']['partInfo']['specificationList'], {}
        for specification in list_specification:
            key = specification['name'].strip()
            if key in dict_count:
                dict_count[key] += 1
            else:
                dict_count[key] = 1

        dict_specification, dict_no = {}, {}
        for i, specification in enumerate(list_specification):
            key = specification['name'].strip()
            if dict_count[key] == 1:
                dict_specification[str(i)] = {key: specification['desc'].strip()}
            else:
                if key in dict_no:
                    dict_no[key] += 1
                else:
                    dict_no[key] = 1
                dict_specification[str(i)] = {f'{key}-{dict_no[key]}': specification['desc'].strip()}

        # = = = = = = = = = = = = = = =

        price = dict_['partNumber']['partInfo']['priceInfo']['price'].strip()

        # = = = = = = = = = = = = = = =

        description_2 = dict_['partNumber']['partInfo']['baseInfoList'][0]['desc'].strip()

        # = = = = = = = = = = = = = = =

        dict_src, list_src = {}, [f'''https://www.hondapartsnow.com{src['largeImg'].strip()}''' for src in dict_['partNumber']['imageList']]
        for i, src in enumerate(list_src):
            dict_src[str(i)] = src

        # = = = = = = = = = = = = = = =

        list_head, list_row = [fit['text'].strip() for fit in dict_['partNumber']['fitVehicleTitleList']], dict_['partNumber']['fitVehicleList']

        # = = = = = = = = = = = = = = =

        url = dict_['partNumber']['canonical'].strip()

        # = = = = = = = = = = = = = = =

        fit = dict_['partNumber']['fitModel'].strip()

        # = = = = = = = = = = = = = = =

        vehicles = '\n'.join(sorted([vehicle['name'].strip() for vehicle in dict_['partNumber']['pnSeo']['vehicles'][0]['links']]))

        # = = = = = = = = = = = = = = =

        list_dict_correct_temp = [{'No': crawler_series['No'],
                                   'Part Number': part_number,
                                   'Type': type_,
                                   'Title': title,
                                   'Subtitle': subtitle,
                                   'SKU': sku,
                                   'Price': price,
                                   'Description 1': description,
                                   'Description 2': description_2,
                                   'Fit': fit,
                                   'Vehicles': vehicles,
                                   'Picture': '',
                                   'Url': url,
                                   'Json_Src': json.dumps(dict_src),
                                   'Json_Specification': json.dumps(dict_specification),
                                   'Row': i + 1} for i in range(len(list_row))]

        # = = = = = = = = = = = = = = =

        for i, row in enumerate(list_row):
            for j, head in enumerate(list_head):
                list_dict_correct_temp[i][head] = row[j]

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend(list_dict_correct_temp)

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No', 'Row'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/1.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/part_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：117

[False] - [请求1次] - [剩余116条] - [2024/11/20 14:04:44] - 23. https://www.hondapartsnow.com/genuine/90666SA0003.html

[False] - [请求1次] - [剩余115条] - [2024/11/20 14:04:44] - 5. https://www.hondapartsnow.com/genuine/91561S84A00M1.html

[True] - [请求1次] - [剩余114条] - [2024/11/20 14:04:44] - 14. https://www.hondapartsnow.com/genuine/90654SA4003.html

[True] - [请求1次] - [剩余113条] - [2024/11/20 14:04:44] - 16. https://www.hondapartsnow.com/genuine/90657SA5003.html

[True] - [请求1次] - [剩余112条] - [2024/11/20 14:04:44] - 19. https://www.hondapartsnow.com/genuine/91568SR2003.html

[True] - [请求1次] - [剩余111条] - [2024/11/20 14:04:44] - 80. https://www.hondapartsnow.com/genuine/91504TBAA01.html

[False] - [请求1次] - [剩余110条] - [2024/11/20 14:04:44] - 86. https://www.hondapartsnow.com/genuine/6786920030.html

[True] - [请求1次] - [剩余109条] - [2024/11/20 14:04:44] - 46. https://www.hondapartsnow.com/genuine/90667TC0U01ZA.html

[True] - [请求1次] - [剩余108条] - [2024/11/20 14:04:44] - 26. https://www.hondapartsnow

C:\Users\Administrator\Desktop\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
